# EthicML

## Runnning experiments on the Adult dataset

### Installation

First we need to install EthicML. Currently, the toolkit isn't on PyPi, but this will change soon.

For now, the toolkit has to be cloned, then installed as an editable package
```
cd <Location to clone to>
git clone https://github.com/predictive-analytics-lab/EthicML.git
cd EthicML
pip install --editable ./
```

(Obviously this notebook is within the package, so we can skip this step)

### Loading the data

EthicML includes some often used datasets from fairness literature.
First, we load one of these... in this example we load the UCI Adult dataset

In [1]:
from ethicml.utility import DataTuple
from ethicml.data.load import load_data
from ethicml.data import Adult, Compas, Credit, German, Sqf, Toy

data: DataTuple = load_data(Adult())
assert (45222, 101) == data.x.shape
assert (45222, 1) == data.s.shape
assert (45222, 1) == data.y.shape

This loads the dataset as a DataTuple, which comprises $x$ (features), $s$ (sensitive attribute and $y$ (class label). Each member of the DataTuple is stored as a Pandas DataFrame.

By default, the Adult dataset uses the binary attribute `sex_Male` as the sensitive feature.

In [2]:
data.s.head()

,sex_Male
0,1
1,1
2,1
3,1
4,0


If we want to run experiments using race as the sensitive attribute we could change that manually, or, as this is a common task, EthicML can split the data for you.

In [3]:
data: DataTuple = load_data(Adult(split="Race"))
assert (45222, 98) == data.x.shape
assert (45222, 5) == data.s.shape
assert (45222, 1) == data.y.shape

In [4]:
data.s.head()

,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,1,0,0
4,0,0,0,0,1


However, we're going to be repeating some of the experiments from FairGP. In that paper they do experiments with race as the sensitive attribute, but the value is binary. The value of race is White or Not_White.

Fortunately, race has been one-hot-encoded so to replicate this we can just drop the features from the sensitive attribute that aren't `race_White`.

The Dataset class is really just a guide that tells EthicML how to read the underlying CSV. So to remove the other race attributes, we can just not include them in our list of sensitive attribute columns.

In [5]:
dataset = Adult("Race")
dataset.sens_attrs = ["race_White"]
data = load_data(dataset)

In [6]:
data.s.head()

,race_White
0,1
1,1
2,1
3,0
4,1


### Evaluating some models

In [9]:
from ethicml.algorithms.inprocess import Agarwal, InAlgorithm, LR, SVM, Kamishima, Kamiran
from ethicml.algorithms.preprocess import Upsampler
from ethicml.metrics import Accuracy, CV, TPR, ProbPos
from ethicml.evaluators import evaluate_models

datasets = [dataset, Toy()]
preprocess_models = [Upsampler()]
# inprocess_models = [Agarwal(), Kamishima(), LR(), SVM(kernel='linear'), Kamiran()]
inprocess_models = [LR(), SVM(kernel='linear'), Kamiran()]
postprocess_models = []
metrics = [Accuracy(), CV()]
per_sens_metrics = [Accuracy(), TPR(), ProbPos()]

In [10]:
test123 = evaluate_models(datasets, preprocess_models, inprocess_models, postprocess_models, metrics, per_sens_metrics, test_mode=False, repeats=2)

100%|██████████| 28/28 [00:26<00:00,  3.33it/s, task=Kamiran & Calders LR, dataset=Toy, transform=Upsample, repeat=1]      


In [12]:
test123

Accuracy        CV  \
dataset transform    model                repeat                       
Adult   no_transform Agarwal LR           0-2410  0.851410  0.916525   
                     Kamishima            0-2410  0.849420  0.905814   
                     Logistic Regression  0-2410  0.850304  0.928949   
                     SVM                  0-2410  0.861913  0.938460   
                     Kamiran & Calders LR 0-2410  0.850525  0.926266   
                     Agarwal LR           0-2410  0.851741  0.918604   
                     Logistic Regression  0-2410  0.850415  0.928155   
                     SVM                  0-2410  0.851962  0.914970   
                     Kamiran & Calders LR 0-2410  0.850636  0.930913   
        Upsample     Agarwal LR           0-2410  0.799005  0.865892   
                     Logistic Regression  0-2410  0.789718  0.851146   
                     SVM                  0-2410  0.793919  0.857253   
                     Kamiran & Calders LR 0-2410  0.788834  0.851298   
        no_transform Agarwal LR           0-2410  0.851410  0.916525   
                     Kamishima            0-2410  0.849420  0.905814   
                     Logistic Regression  0-2410  0.850304  0.928949   
                     SVM                  0-2410  0.861913  0.938460   
                     Kamiran & Calders LR 0-2410  0.850525  0.926266   
                     Agarwal LR           0-2410  0.851741  0.918604   
                     Logistic Regression  0-2410  0.850415  0.928155   
                     SVM                  0-2410  0.851962  0.914970   
                     Kamiran & Calders LR 0-2410  0.850636  0.930913   
                     Agarwal LR           0-2410  0.851410  0.916525   
                     Kamishima            0-2410  0.849420  0.905814   
                     Logistic Regression  0-2410  0.850304  0.928949   
                     SVM                  0-2410  0.861913  0.938460   
                     Kamiran & Calders LR 0-2410  0.850525  0.926266   
                     Agarwal LR           0-2410  0.851741  0.918604   
                     Logistic Regression  0-2410  0.850415  0.928155   
                     SVM                  0-2410  0.851962  0.914970   
...                                                    ...       ...   
        Upsample     SVM                  0-2410  0.793919  0.857253   
                     Kamiran & Calders LR 0-2410  0.788502  0.851169   
        no_transform Agarwal LR           1-4820  0.843007  0.928962   
                     Logistic Regression  1-4820  0.841902  0.931427   
                     SVM                  1-4820  0.843449  0.929112   
                     Kamiran & Calders LR 1-4820  0.844002  0.932603   
                     Logistic Regression  0-2410  0.850415  0.928155   
                     SVM                  0-2410  0.851962  0.914970   
                     Kamiran & Calders LR 0-2410  0.850415  0.930656   
        Upsample     Logistic Regression  0-2410  0.788502  0.851169   
                     SVM                  0-2410  0.793919  0.857253   
                     Kamiran & Calders LR 0-2410  0.789608  0.851017   
        no_transform Logistic Regression  1-4820  0.841902  0.931427   
                     SVM                  1-4820  0.843449  0.929112   
                     Kamiran & Calders LR 1-4820  0.844002  0.932603   
        Upsample     Logistic Regression  1-4820  0.752570  0.857605   
                     SVM                  1-4820  0.793035  0.863390   
                     Kamiran & Calders LR 1-4820  0.752570  0.857605   
Toy     no_transform Logistic Regression  0-7230  0.887500  0.609302   
                     SVM                  0-7230  0.887500  0.609302   
                     Kamiran & Calders LR 0-7230  0.877500  0.659666   
        Upsample     Logistic Regression  0-7230  0.877500  0.659666   
                     SVM                  0-7230  0.877500  0.659666   
                  

In [11]:
test123.groupby(level=[0,1,2]).agg(['mean', 'std'])[['Accuracy', 'Accuracy_race_White_0/race_White_1', 'TPR_race_White_0/race_White_1', 'prob_pos_race_White_0/race_White_1']]

Accuracy            \
                                               mean       std   
dataset transform    model                                      
Adult   Upsample     Agarwal LR            0.798355  0.001289   
                     Kamiran & Calders LR  0.779580  0.015777   
                     Logistic Regression   0.780228  0.016157   
                     SVM                   0.793694  0.000385   
        no_transform Agarwal LR            0.850180  0.003217   
                     Kamiran & Calders LR  0.849458  0.002452   
                     Kamishima             0.849420  0.000000   
                     Logistic Regression   0.849005  0.003208   
                     SVM                   0.853933  0.006490   
Toy     Upsample     Agarwal LR            0.849167  0.011277   
                     Kamiran & Calders LR  0.875893  0.003087   
                     Logistic Regression   0.875893  0.003087   
                     SVM                   0.875893  0.003087   
        no_transform Agarwal LR            0.829219  0.032032   
                     Kamiran & Calders LR  0.876019  0.002287   
                     Logistic Regression   0.877130  0.016007   
                     SVM                   0.880093  0.011433   

                                          Accuracy_race_White_0/race_White_1  \
                                                                        mean   
dataset transform    model                                                     
Adult   Upsample     Agarwal LR                                     0.928318   
                     Kamiran & Calders LR                           0.915295   
                     Logistic Regression                            0.915456   
                     SVM                                            0.925306   
        no_transform Agarwal LR                                     0.940825   
                     Kamiran & Calders LR                           0.941908   
                     Kamishima                                      0.949520   
                     Logistic Regression                            0.943269   
                     SVM                                            0.943321   
Toy     Upsample     Agarwal LR                                          NaN   
                     Kamiran & Calders LR                                NaN   
                     Logistic Regression                                 NaN   
                     SVM                                                 NaN   
        no_transform Agarwal LR                                          NaN   
                     Kamiran & Calders LR                                NaN   
                     Logistic Regression                                 NaN   
                     SVM                                                 NaN   

                                                     \
                                                std   
dataset transform    model                            
Adult   Upsample     Agarwal LR            0.006361   
                     Kamiran & Calders LR  0.005510   
                     Logistic Regression   0.005604   
                     SVM                   0.003082   
        no_transform Agarwal LR            0.000616   
                     Kamiran & Calders LR  0.001486   
                     Kamishima             0.000000   
                     Logistic Regression   0.001007   
                     SVM                   0.004103   
Toy     Upsample     Agarwal LR                 NaN   
                     Kamiran & Calders LR       NaN   
                     Logistic Regression        NaN   
                     SVM                        NaN   
        no_transform Agarwal LR                 NaN   
                     Kamiran & Calders LR       NaN   
                     Logistic Regression        NaN   
                     SVM                        NaN   

                                        

In [9]:
from ethicml.algorithms.inprocess import Agarwal, InAlgorithm, LR, SVM, Kamishima
from ethicml.algorithms.preprocess import Kamiran
from ethicml.metrics import Accuracy, CV, TPR, ProbPos
from ethicml.evaluators.evaluate_models import evaluate_models

datasets = [Adult()]
preprocess_models = []
inprocess_models = [Agarwal(), Kamishima(), LR(), SVM()]
postprocess_models = []
metrics = [Accuracy(), CV()]
per_sens_metrics = [Accuracy(), TPR(), ProbPos()]
test123 = evaluate_models(datasets, preprocess_models, inprocess_models, postprocess_models, metrics, per_sens_metrics, test_mode=False, repeats=10)

 98%|█████████▊| 50/51 [41:59<00:50, 50.40s/it]


In [13]:
test123.groupby(level=[0,1,2]).agg(['mean', 'std'])[['Accuracy', 'Accuracy_sex_Male_0/sex_Male_1', 'TPR_sex_Male_0/sex_Male_1', 'prob_pos_sex_Male_0/sex_Male_1']]


Accuracy            \
                                              mean       std   
dataset transform    model                                     
Adult   no_transform Agarwal              0.848572  0.003176   
                     Kamishima            0.850384  0.002961   
                     Logistic Regression  0.850097  0.003253   
                     SVM                  0.864725  0.002844   

                                         Accuracy_sex_Male_0/sex_Male_1  \
                                                                   mean   
dataset transform    model                                                
Adult   no_transform Agarwal                                   1.139960   
                     Kamishima                                 1.137531   
                     Logistic Regression                       1.138135   
                     SVM                                       1.117742   

                                                   TPR_sex_Male_0/sex_Male_1  \
                                               std                      mean   
dataset transform    model                                                     
Adult   no_transform Agarwal              0.006937                  0.931315   
                     Kamishima            0.007479                  0.815207   
                     Logistic Regression  0.007381                  0.825997   
                     SVM                  0.006871                  0.757971   

                                                    \
                                               std   
dataset transform    model                           
Adult   no_transform Agarwal              0.086210   
                     Kamishima            0.037357   
                     Logistic Regression  0.034157   
                     SVM                  0.044756   

                                         prob_pos_sex_Male_0/sex_Male_1  \
                                                                   mean   
dataset transform    model                                                
Adult   no_transform Agarwal                                   0.354343   
                     Kamishima                                 0.291610   
                     Logistic Regression                       0.297058   
                     SVM                                       0.260670   

                                                    
                                               std  
dataset transform    model                          
Adult   no_transform Agarwal              0.040643  
                     Kamishima            0.013690  
                     Logistic Regression  0.014001  
                     SVM                  0.018487